In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.tag as tag
import thor.visualize as visualize

notebook_name = "cpol_demo.ipynb"
__file__ = "/home/ewan/Documents/THOR/examples/cpol_demo.ipynb"


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
# Parent directory for saving outputs
base_local = Path("/home/ewan/THOR_output")
start = "2005-11-13T19:00:00"
end = "2005-11-13T20:00:00"

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
cpol_options = data.aura.cpol_data_options(
    start=start, end=end, converted_options=converted_options
)
era5_pl_options = data.era5.data_options(start=start, end=end)
era5_sl_options = data.era5.data_options(
    start=start, end=end, data_format="single-levels"
)
data_options = option.consolidate_options(
    [cpol_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, filename="cpol_era5")

# Create the grid_options dictionary using the first file in the cpol dataset
filepath = data_options["cpol"]["filepaths"][0]
with xr.open_dataset(filepath) as cpol:
    if converted_options["load"]:
        latitude, longitude = [
            cpol.latitude.values.tolist(),
            cpol.longitude.values.tolist(),
        ]
    else:
        lats = cpol["point_latitude"].isel(z=0).values
        lons = cpol["point_longitude"].isel(z=0).values
        latitude, longitude = grid.new_geographic_grid(lats, lons, 0.025, 0.025)
grid_options = grid.create_options(
    name="geographic", latitude=latitude, longitude=longitude
)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, filename="cpol_geographic")

# Create the tag_options dictionary
era5_pl_tag_options = data.era5.tag_options()
era5_sl_tag_options = data.era5.tag_options(dataset="era5_sl")
tag_options = option.consolidate_options([era5_pl_tag_options, era5_sl_tag_options])
tag.save_tag_options(tag_options, filename="era5")

# Create the track_options dictionary
track_options = option.mcs(dataset="cpol", tags=["era5_pl", "era5_sl"])
option.save_track_options(track_options, filename="cpol_mcs")

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(
        obj,
        save=True,
        style="presentation",
    )
    for obj in ["cell", "middle_cloud", "anvil", "mcs"]
}
visualize.option.save_display_options(visualize_options, filename="runtime_mcs")

2024-06-30 10:21:41,432 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/data_options/cpol_era5.yaml


2024-06-30 10:21:41,561 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/grid_options/cpol_geographic.yaml
2024-06-30 10:21:41,606 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/tag_options/era5.yaml
2024-06-30 10:21:41,609 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/track_options/cpol_mcs.yaml
2024-06-30 10:21:41,619 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/visualize_options/runtime_mcs.yaml


In [3]:
# visualize_options['mcs']["figures"]["mask"]["member_objects"] = ["cell", "anvil"]
# visualize_options['mcs']["figures"]["mask"]["member_levels"] = [0, 0]
visualize_options['mcs']["figures"]["mask"]["figsize"] = (12, 3.5)
visualize_options['mcs']["figures"]["mask"]["wspace"] = 0.05

In [4]:
visualize_options['mcs']["figures"]["mask"]

{'style': 'presentation', 'figsize': (12, 3.5), 'wspace': 0.05}

In [15]:
times = data.aura.generate_cpol_times(data_options["cpol"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    tag_options,
    visualize_options,
    output_directory=base_local / "runs/cpol_demo",
)

2024-06-30 10:41:38,103 - thor.track - INFO - Processing 2005-11-13T19:00:08.
2024-06-30 10:41:38,105 - thor.data.aura - DEBUG - Updating cpol dataset for 2005-11-13T19:00:08.
2024-06-30 10:41:38,106 - thor.data.aura - DEBUG - Converting cpol data from twp10cpolgrid150.b2.20051113.190000.nc
2024-06-30 10:41:38,842 - thor.track - DEBUG - Processing hierarchy level 0.
2024-06-30 10:41:38,843 - thor.track - DEBUG - Tracking cell.
2024-06-30 10:41:40,385 - thor.track - DEBUG - Tracking middle_cloud.
2024-06-30 10:41:41,714 - thor.track - DEBUG - Tracking anvil.
2024-06-30 10:41:43,046 - thor.track - DEBUG - Processing hierarchy level 1.
2024-06-30 10:41:43,047 - thor.track - DEBUG - Tracking mcs.
2024-06-30 10:41:46,703 - thor.data.era5 - DEBUG - Updating era5_pl dataset for 2005-11-13T19:00:08.
2024-06-30 10:41:46,705 - thor.data.era5 - DEBUG - Subsetting u_era5_oper_pl_20051101-20051130.nc
2024-06-30 10:41:46,765 - thor.data.era5 - DEBUG - Subsetting v_era5_oper_pl_20051101-20051130.nc
2

In [13]:
data_options["cpol"]

{'name': 'cpol',
 'start': '2005-11-13T19:00:00',
 'end': '2005-11-13T20:00:00',
 'parent_remote': 'https://dapds00.nci.org.au/thredds/fileServer/hj10',
 'save_local': False,
 'parent_local': '/home/ewan/THOR_output/input_data/raw',
 'converted_options': {'save': True, 'load': False, 'parent_converted': None},
 'filepaths': ['/home/ewan/THOR_output/input_data/raw/cpol/cpol_level_1b/v2020/gridded/grid_150km_2500m/2005/20051113/twp10cpolgrid150.b2.20051113.190000.nc',
  '/home/ewan/THOR_output/input_data/raw/cpol/cpol_level_1b/v2020/gridded/grid_150km_2500m/2005/20051113/twp10cpolgrid150.b2.20051113.191000.nc',
  '/home/ewan/THOR_output/input_data/raw/cpol/cpol_level_1b/v2020/gridded/grid_150km_2500m/2005/20051113/twp10cpolgrid150.b2.20051113.192000.nc',
  '/home/ewan/THOR_output/input_data/raw/cpol/cpol_level_1b/v2020/gridded/grid_150km_2500m/2005/20051113/twp10cpolgrid150.b2.20051113.193000.nc',
  '/home/ewan/THOR_output/input_data/raw/cpol/cpol_level_1b/v2020/gridded/grid_150km_2500m/